In [3]:
import cv2
import numpy as np
import pytesseract
import grip
import time

cap = cv2.VideoCapture("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/SampleVid.mp4")
REFERENCE_INSPECTION_FRAME = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/blankFrame.jpg")
CROPPED_REFERENCE = cv2.imread("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/RefenceUnderJoystick.jpg")
INSPECTION_FRAME_SIMILARITY_THRESHOLD = 60

cv2.startWindowThread()

def isInspectionMode(frame):
    difference = cv2.absdiff(REFERENCE_INSPECTION_FRAME, frame)
    res = difference.astype(np.uint8)
    percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))

    if percentage >= INSPECTION_FRAME_SIMILARITY_THRESHOLD:
        return True
    else:
        return False
    
def isInspectionClick(frame):
    cropped_frame_left = frame[550:650, 20:300]
    cropped_frame_right = frame[550:650, 990:1270]
    if isInspectionMode(cropped_frame):
        time_frame = frame[10:75, 600:700]
        time_str = pytesseract.image_to_string(time_frame, config='--psm 6')

        difference = cv2.absdiff(CROPPED_REFERENCE, cropped_frame)
        res = difference.astype(np.uint8)
        percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))

        if percentage < 100:
            return True
        else:
            return False
    else:
        print ("Not an Inspection Frame")
        return False
    
def flightTime(frame):
    time_frame = frame[10:75, 600:700]
    time_str = pytesseract.image_to_string(time_frame, config='--psm 6')
    try:
        seconds = (time.strptime(time_str, '%M:%S')[4]*60) + time.strptime(time_str, '%M:%S')[5]
        return seconds
    except:
        return
    
def checkDifference(frame):
    print (frame.shape, CROPPED_REFERENCE.shape)
    difference = cv2.subtract(frame, CROPPED_REFERENCE)
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(difference, low_threshold, high_threshold)
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 200  # minimum number of pixels making up a line
    max_line_gap = 200  # maximum gap in pixels between connectable line segments
    line_image = np.copy(frame) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)

    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)
    
    # Draw the lines on the  image
    return cv2.addWeighted(img, 0.8, line_image, 1, 0)
    
    

In [ ]:
count = 0

if cap.isOpened(): # Check if camera opened successfully
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            difference = cv2.absdiff(REFERENCE_INSPECTION_FRAME, frame)
            res = difference.astype(np.uint8)
            percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))
            
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, str(percentage),(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.0,(255,0,255),lineType=cv2.LINE_AA)
            # cv2.imshow("Video", frame)
            
            if percentage >= INSPECTION_FRAME_SIMILARITY_THRESHOLD:
                
            
            
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break
            
else:
    print("Error opening video stream or file")

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame = frame[550:650, 0:1280]
        cv2.imshow("cropped frame", cropped_frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

In [ ]:
frame_width = int( CROPPED_REFERENCE.shape[0])
frame_height =int( CROPPED_REFERENCE.shape[1])
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/InspectionMode.avi', fourcc, 1, (frame_height, frame_width))
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame = frame[550:650, 0:1280]
        if isInspectionMode(cropped_frame):
            out.write(cropped_frame)
            print ("Added frame to video.")
        else:
            print ("Skipped frame")
    else:
        break
print ("Done.")

In [ ]:
count = 0
# Extracted Seconds from App
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame = frame[550:650, 0:1280]
        if isInspectionMode(cropped_frame):
            time_frame = frame[10:75, 600:700]
            time_str = pytesseract.image_to_string(time_frame, config='--psm 6')
            
            difference = cv2.absdiff(CROPPED_REFERENCE, cropped_frame)
            res = difference.astype(np.uint8)
            percentage = abs(100 - ((np.count_nonzero(res) * 100)/ res.size))
            
            cv2.putText(frame, "%s - percentage. %s - time" % (percentage, time_str), (0, 0), cv2.FONT_HERSHEY_SIMPLEX, 1.0,(255,0,255),lineType=cv2.LINE_AA)
            # cv2.imshow("time", time_frame)
            # cv2.imshow("frame", frame)
            if percentage < 100:
                print ("Writing frame %d to disk with percent similarity of %d" % (count, percentage))
                cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/frame%d.jpg" % count, frame)
                count += 1
            print (percentage, count)
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
print("Done.")

In [ ]:
count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        if isInspectionClick(frame):
            cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/InspectionClickFrame%d.jpg" % count, frame)
            print (flightTime(frame))
            count += 1

In [4]:
count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret == True:
        cropped_frame_left = frame[550:650, 20:300]
        cropped_frame_right = frame[550:650, 990:1270]
        time_frame = frame[10:75, 600:700]
        if isInspectionMode(frame):
            cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/Lines%d.jpg" % count, checkDifference(cropped_frame_left))
            count += 1
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.imwrite("/Users/akshar/Downloads/Humans in Autonomy/DroneFootageAnalysis/RefenceUnderJoystick.jpg", cropped_frame)

(100, 280, 3) (100, 1280, 3)


error: OpenCV(3.4.2) /opt/concourse/worker/volumes/live/9523d527-1b9e-48e0-7ed0-a36adde286f0/volume/opencv-suite_1535558719691/work/modules/core/src/arithm.cpp:659: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'
